In [1]:
import logging
import warnings
from pprint import pprint

import numpy as np
from openeye import oechem
from openff.qcsubmit.common_structures import TorsionIndexer
from openff.qcsubmit.factories import TorsiondriveDatasetFactory
from openforcefield.topology import Molecule
from tqdm import tqdm

In [2]:
# Warnings that tell us we have undefined stereo and charged molecules
logging.getLogger("openforcefield").setLevel(logging.ERROR)
warnings.simplefilter("ignore")

# Dataset Preperation

Load in the SMILES patterns of the molecules to include.

In [3]:
with open("dataset.smi") as file:
    smiles_patterns = file.read().split("\n")

Define the SMARTS pattern which will match *amide torsions.

In [4]:
torsion_smarts = "[*:1]-[NX3+0:2]-[C:3]=[O,S,N+0:4]"

Load in the molecules and flag the torsion(s) to be driven.

In [5]:
molecules = []

for smiles_pattern in tqdm(smiles_patterns):

    molecule: Molecule = Molecule.from_smiles(smiles_pattern)

    try:
        molecule.generate_conformers(n_conformers=1)
    except Exception:
        print(f"Skipping {smiles_pattern} - OMEGA error.")
        continue

    matches = molecule.chemical_environment_matches(torsion_smarts)

    torsion_indexer = TorsionIndexer()

    for match in matches:
        torsion_indexer.add_torsion(match, (-180, 180), True)

    if torsion_indexer.n_torsions == 0:
        print(f"Skipping {smiles_pattern} - no matches.")

    molecule.properties["dihedrals"] = torsion_indexer
    molecules.append(molecule)

100%|██████████| 260/260 [00:42<00:00,  6.05it/s]


Prepare the main dataset from the molecule list.

In [6]:
dataset_factory = TorsiondriveDatasetFactory()

dataset = dataset_factory.create_dataset(
    dataset_name="OpenFF Amide Torsion Set v1.0",
    tagline="Amides, thioamides and amidines diversely functionalized.",
    description="This dataset contains a set of 'simple' amides, thioamides, and amidines which have been "
    "functionalised with groups of varying electron-donating and electron-withdrawing characteristics. It was curated "
    "as part of an effort to explore why the cis-trans preference of amides is poorly reproduced in OpenFF 1.3.0, and "
    "as an extra source of data to try and remedy this.",
    molecules=molecules,
)

dataset.metadata.submitter = "simonboothroyd"
dataset.metadata.long_description_url = (
    "https://github.com/openforcefield/qca-dataset-submission/tree/master/"
    "submissions/"
    "2021-03-23-OpenFF-Amide-Torsion-Set-v1.0"
)

dataset.provenance["constructure"] = "0.0.1"

Preparation                   : 100%|█████████| 214/214 [00:02<00:00, 88.57it/s]


Make sure the molecules in the dataset match the input molecules

In [7]:
old_smiles = {Molecule.from_smiles(smiles).to_smiles(isomeric=False) for smiles in smiles_patterns}
new_smiles = {molecule.to_smiles(isomeric=False) for molecule in dataset.molecules}

assert len(old_smiles.symmetric_difference(new_smiles)) == 0

Describe the molecule in the dataset

In [8]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of torsion drives         ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Number of unique molecules        214
Number of filtered molecules      0
Number of torsion drives          260
Number of conformers min mean max 1   1.26 2
Mean molecular weight: 154.54
Max molecular weight: 317.32
Charges: [-2.0, -1.0, 0.0]


Describe the dataset

In [9]:
pprint(dataset.metadata.dict())

{'collection_type': 'TorsiondriveDataset',
 'creation_date': datetime.date(2021, 3, 24),
 'dataset_name': 'OpenFF Amide Torsion Set v1.0',
 'elements': {'H', 'O', 'S', 'C', 'N'},
 'long_description': "This dataset contains a set of 'simple' amides, "
                     'thioamides, and amidines which have been functionalised '
                     'with groups of varying electron-donating and '
                     'electron-withdrawing characteristics. It was curated as '
                     'part of an effort to explore why the cis-trans '
                     'preference of amides is poorly reproduced in OpenFF '
                     '1.3.0, and as an extra source of data to try and remedy '
                     'this.',
 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-03-23-OpenFF-Amide-Torsion-Set-v1.0', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submi

In [10]:
for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    pprint(obj.dict())

Spec: default
{'basis': 'DZVP',
 'implicit_solvent': None,
 'method': 'B3LYP-D3BJ',
 'program': 'psi4',
 'spec_description': 'Standard OpenFF optimization quantum chemistry '
                     'specification.',
 'spec_name': 'default',
 'store_wavefunction': 'none'}


In [11]:
pprint(dataset.scf_properties)

[<SCFProperties.Dipole: 'dipole'>,
 <SCFProperties.Quadrupole: 'quadrupole'>,
 <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>,
 <SCFProperties.MayerIndices: 'mayer_indices'>]


Export the dataset.

In [12]:
dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file("dataset.smi", "smi")

dataset.visualize("dataset.pdf", columns=8)